# Basic Workflow for R
---------------------------------------

This jupyter-notebook contains basic R workflow for network visualizatin.

You can understand the basic workflow by reading this, but the RCy3's documentation is really useful and you can know much more about this.

The documentation : https://www.bioconductor.org/packages/release/bioc/html/RCy3.html

# Setup
---------------------------------------
To execute this, please sutisfy following items.

- Java SE 8
- Cytoscape version 3.3+
- CyREST
- Use docker file to do this. The docker file provide you the environment below.
    - RCy3
    - igraph
    - ...

# Start a new session
---------------------------------------
To start a new session, you should do the following step.

### Steps
1. Please execute cytoscape
1. Import libraries
1. Load Data
1. Shaping data

# IMPORTANT NOTE

If you are running this notebook from a Docker container, you need to specify ***IP address of your machine running Cytoscape***.  Usually, it is the IP address of your laptop/workstation.  Please edit the following line before you run this notebook!

```
ip_address <- '192.168.1.1'
```

In [ ]:
# !!!!!!!!!!!! EDIT  THIS VALUE FIRST!!!!!!!!!!!!!!
ip_address <- 'IP_OF_YOUR_MACHINE'
ip_address <- '137.110.137.158'

library(RCy3)
library(igraph)
library(RJSONIO)
library(httr)
library(data.table)


# Test Connection to Cytoscape
conn <- CytoscapeConnection(host = ip_address)
print(conn)

# Start from scratch: Delete all networks
deleteAllWindows(conn)

## Import network table from Data Frame object

In this example, we'll use [HumanNet v1]() data sets as the base network.  Also, for convenience, we'll use [data.table](https://github.com/Rdatatable/data.table/wiki) library to import data.

In [ ]:
# Read as an igraph object
humannet.url <- "http://www.functionalnet.org/humannet/HumanNet.v1.benchmark.txt"
humannet.ig <- read.graph(humannet.url)
summary(humannet.ig)

In [ ]:
cw <- CytoscapeWindow('vignette', host = ip_address , graph=g, overwrite=TRUE)

In [ ]:
# Load Data
gal.table <- read.table('sampleData/galFiltered.sif',stringsAsFactors=FALSE)
head(gal.table)

In [ ]:
# create graph class
g <- new ('graphNEL', edgemode='directed')

# Get NodesVec
gal.table.nodevec <- unique(c(gal.table[[1]], gal.table[[3]]))

# add nodes to graph
for(node in gal.table.nodevec){
    g <- graph::addNode(node, g)
}

# get EdgeList
gal.table.fromvec = gal.table[[1]]
gal.table.tovec = gal.table[[3]]

for (index in 1:length(gal.table.fromvec)){
    g <- graph::addEdge (gal.table.fromvec[[index]] ,gal.table.tovec[[index]], g)
}

In [ ]:
# show it in cytescape
cw <- CytoscapeWindow('vignette', host = ip_address , graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')

### Option 2: Load from list of URLs

Instead of passing actual graph data, you can pass list of file locations to load files into Cytoscape.

#### Note:
This feature is not implemented in RCy3, so you need to use the following code snippet to do this...

In [ ]:
port.number = 1234
base.url = paste("http://", ip_address, ":", toString(port.number), "/v1/", sep="")
res <-GET(base.url)
rawToChar(res$content)

In [ ]:
network.url = paste(base.url, "networks?source=url&collection=FromR", sep="")

urls <-c(
    'file:///Users/kono/git/cyrest-examples/notebooks/cookbook/sampleData/galFiltered.sif',
    'file:///Users/kono/git/cyrest-examples/notebooks/cookbook/sampleData/galFiltered.gml'
)

In [ ]:
res <- POST(url=network.url, body=toJSON(urls), encode="json")
network.suid = unname(fromJSON(rawToChar(res$content)))
print(network.suid)

Then, you can get this image in cytoscape.

![cytoscape image](graph_image.jpeg)

# Upload  a node attribute table.

---------------------------------------

## Import Annotation Data from bioconductor and merge it into Network data
---------------------------------------

In this example of workflow, by executing the following steps, you can load annotation data and merge it.

1. First, import network data from local file/URL to Cytoscape.
1. Second, import annotation data from database(in R, we can use bioconductor as database).
  - Human database in bioconductor:
  - https://www.bioconductor.org/packages/release/data/annotation/html/org.Hs.eg.db.html
  - Yeast database in bioconductor :
  - https://www.bioconductor.org/packages/release/data/annotation/html/org.Sc.sgd.db.html
1. Finally, merge the above two data table and push it to Cytoscape.

Now, we use the data file 'galFiltered.sif' that is the yeast data. So, we will try to add annotation data to this.


In [ ]:
# Second, import annotation data from database(in R, we can use bioconductor as database).
# import library to access database
library(org.Sc.sgd.db)

# import Data
# DESCRIPTION data
descriptions <- select(org.Sc.sgd.db, keys=getAllNodes(cw), columns="DESCRIPTION")
df.descriptions <- data.frame(descriptions)

# GENENAME data
gene.names <- select(org.Sc.sgd.db, keys=getAllNodes(cw), columns="GENENAME")
df.gene.names <- data.frame(gene.names)

# GO data
go <- select(org.Sc.sgd.db, keys=getAllNodes(cw), columns="GO")
df.go <- data.frame(go)

# Finally, merge the above two data table and push it to Cytoscape.
# set DESCRIPTION as attribute data
nodeDataDefaults(g, attr = "DESCRIPTION") <- "undefined"
attr(nodeDataDefaults(g, attr = "DESCRIPTION"), "class") <- "STRING"

# set GENENAME as attribute data
nodeDataDefaults(g, attr = "name") <- "undefined"
attr(nodeDataDefaults(g, attr = "name"), "class") <- "STRING"

# set GO as attribute data
#nodeDataDefaults(g, attr = "go") <- "undefined"
#attr(nodeDataDefaults(g, attr = "go"), "class") <- "LIST"

# marge DESCRIPTION attribute.
for (index in 1:length(df.descriptions$DESCRIPTION)){
    nodeData(g, df.descriptions$ORF[[index]], "DESCRIPTION") <- df.descriptions$DESCRIPTION[[index]]
}

# marge GENENAME attribute.
for (index in 1:length(df.gene.names$GENENAME)){
    nodeData(g, df.gene.names$ORF[[index]], "name") <- df.gene.names$GENENAME[[index]]
}

# marge GO attribute.
#for (index in 1:length(df.gene.names$GENENAME)){
#    nodeData(g, df.gene.names$ORF[[index]], "name") <- df.gene.names$GENENAME[[index]]
#}

# show it in cytescape
cw <- CytoscapeWindow('vignette', graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')

# Upload an edge attribute table

In [ ]:
# add attribute:"edgeType" and the type:"STRING"
edgeDataDefaults(g, attr = "edgeType") <- "undefined"
attr(edgeDataDefaults(g, attr = "edgeType"), "class") <- "STRING"

# get EdgeList and type
gal.table.fromvec <- gal.table[,1]
gal.table.type <- gal.table[,2]
gal.table.tovec <-  gal.table[,3]

# add attribute value to the network
for (index in 1:length(gal.table.fromvec)){
    edgeData(g, gal.table.fromvec[[index]] ,gal.table.tovec[[index]], "edgeType") <- gal.table.type[[index]]
}
cw <- CytoscapeWindow('vignette', graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')

# Select edges based on some edge attributes
---------------------------------------

### How to select edges based on their attributes?

First, we have to set attributes to edges. Then, we can select edges based on attributes.

### Method : selectEdges

#### Usage
selectEdges(obj, edge.names, preserve.current.selection=TRUE)

#### Arguments

- **obj** : a CytoscapeWindowClass object.  
- **edge.names** : a list of strings, the names of edges to select.  
- **preserve.current.selection** : a logical object.  




In [ ]:
# clear selection
clearSelection (cw)

In [ ]:
# selection Edges
vec = c()
for (index in 1:length(gal.table.type)){
    if(gal.table.type[[index]] == 'pp'){
        vec = c(vec, paste(gal.table.fromvec[[index]], '(pp)', gal.table.tovec[[index]]))
    }
}
selectEdges (cw, vec)

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](select_edge_image.jpeg)

# Hide these edges
---------------------------------------

If you want to hide selected edges, you can use this method.

### Method : hideSelectedEdges

#### Usage

hideSelectedEdges(obj)

#### Arguments

obj a CytoscapeWindowClass object.

In [ ]:
# The code is only this!!
# TODO : this methods have some error. I can't do anything with this method.
hideSelectedEdges (cw)

# Select nodes based on some node attributes
---------------------------------------

### method : selectNodes

#### Usage
selectNodes(obj, node.names, preserve.current.selection=TRUE)

#### Arguments

- **obj** : a CytoscapeWindowClass object.
- **node.names** : a list of strings, the names of nodes to select.
- **preserve.current.selection** : a logical object.


In [ ]:
# get attributes data
gal.node.attrs <- read.table('sampleData/galFiltered_node.attrs',stringsAsFactors=FALSE, skip=1)
head(gal.node.attrs)

In [ ]:
g <- initNodeAttribute (graph=g, attribute.name='moleculeType', attribute.type='char', default.value='undefined')

# get EdgeAttrs
gal.node = gal.node.attrs[[1]]
gal.attrs = gal.node.attrs[[3]]

for (index in 1:length(gal.node)){
    nodeData (g, gal.node, 'moleculeType') <- as.character(gal.attrs[[index]])
}

cw <- setGraph (cw, g)
displayGraph (cw)

In [ ]:
# clear selection
clearSelection (cw)

In [ ]:
vac = c()
for (index in 1:length(gal.node)){
    if(gal.attrs[[index]] == '1'){
        vec = c(vec, gal.node[[index]])
    }
}

selectNodes(cw, vec)

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](select_nodes_image.jpeg)

# Hide these nodes
---------------------------------------
### Method : hideSelectedNodes

#### Usage

hideSelectedNodes(obj)

#### Arguments

obj : a CytoscapeWindowClass object.


In [ ]:
# TODO : There are some error in this method. I can't do anything by this.
hideSelectedNodes(cw)

# Apply a layout the the network that is not hidden
---------------------------------------
### Method : getLayoutNames, layoutNetwork

We use two method in this section. 'getLayoutNames' let us know available layoutNames and we can apply one of them by 'layoutNetwork' method.

#### 1. Usage
getLayoutNames(obj)

#### 1. Arguments
obj : a CytoscapeConnectionClass object.

#### 2. Usage
layoutNetwork(obj, layout.name= grid )

#### 2. Arguments
obj a CytoscapeWindowClass object.
layout.name a string, one of the values returned by getLayoutNames, ’grid’ by default.


In [ ]:
# get available layout names
getLayoutNames(cw)

In [ ]:
# execute layout that you want
layoutNetwork(cw, layout.name= 'isom')

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](isom_graph_image.jpeg)

In [ ]:
# execute layout that you want
layoutNetwork(cw, layout.name= 'kamada-kawai')

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](kamada_kawai_image.jpeg)

# Apply a visual style to this network
---------------------------------------

By default, Cytoscape displays nodes as pale red circles circles, edges as blue undecorated lines, selected nodes as yellow and selected edges as red. RCy3 provides an easy way to change these defaults. More interestingly, RCy3 provides easy programmatic access to the vizmapper, by means of which the size, shape and color of nodes and edges is determined by the data attributes you have attached to those nodes and edges.

## The methods
---------------------------------------

There are so many methods to do this.

#TODO : I have to remove following message when I push. I think this section will be similar to cookbook, so I will copy and modify with it.

At this point I do every bit of the visual style manually, but it is always the same.

That is because I do not know how to transfer a visual style from one session to the next yet.

It would be great if that was automated (I do it manually for one network and then it is automatic for the following ones that have the same exact attributes)'